#### 文献关键词画图



In [23]:
# 导入必要的库
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from collections import Counter, defaultdict
import re
from wordcloud import WordCloud
import matplotlib.colors as mcolors
from matplotlib.font_manager import FontProperties
from pyecharts import options as opts
from pyecharts.charts import Graph
from IPython.display import display, HTML, IFrame
import json
import warnings

warnings.filterwarnings('ignore')

In [4]:
# 设置matplotlib中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [5]:
# 下载nltk词库（如果首次使用）
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')

In [24]:
def load_data(file_path):
    """加载CSV数据文件"""
    df = pd.read_csv(file_path, encoding='utf-8')
    print(f"数据集大小: {df.shape}")
    print(f"数据集列名: {df.columns.tolist()}")
    return df

In [25]:
insight_keywords = ['continuous fibers', 'path planning method', 'robot programming', 'optimization', 'topology optimisation']

In [26]:
file_path = './results/CFpathPlanning101_20250510_17/CFpathPlanning101_replaced_synonyms.csv'
save_path = './results'
df = load_data(file_path)

数据集大小: (101, 26)
数据集列名: ['作者', 'Author full names', '作者 ID', '文献标题', '年份', '来源出版物名称', '卷', '期', '论文编号', '起始页码', '结束页码', '页码计数', '施引文献', 'DOI', '链接', '归属机构', '带归属机构的作者', '摘要', '作者关键字', '索引关键字', '通讯地址', '文献类型', '出版阶段', '开放获取', '来源出版物', 'EID']


In [27]:
base_filename = os.path.basename(file_path)
file_name_without_ext = os.path.splitext(base_filename)[0]
timestamp = datetime.now().strftime("%Y%m%d_%H")
output_dir = f"{save_path}/{file_name_without_ext}_{timestamp}"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"创建输出目录: {output_dir}")


In [16]:
# 定义函数来为特定关键词创建关系图
def create_keyword_graph(df, target_keyword, output_dir, top_related=30, min_co_occurrence=2):
    """
    为目标关键词创建关系图

    参数:
    - df: 包含文献数据的DataFrame
    - target_keyword: 要分析的目标关键词
    - output_dir: 输出文件保存目录
    - top_related: 要包含的相关关键词数量
    - min_co_occurrence: 最小共现次数阈值
    """
    print(f"\n开始为关键词 '{target_keyword}' 创建关系图...")

    # 获取关键词与其他关键词的共现关系
    keyword_relations = defaultdict(Counter)

    # 分析每篇文献中关键词的共现情况
    for _, row in df.iterrows():
        # 获取文献的关键词列表
        keywords = []

        # 从作者关键字和索引关键字中提取
        for col in ['作者关键字', '索引关键字']:
            if col in df.columns and isinstance(row[col], str) and row[col].strip():
                # 分割关键词
                kws = [k.strip().lower() for k in re.split(r'[;,，；]', row[col]) if k.strip()]
                keywords.extend(kws)

        # 去重
        keywords = list(set(keywords))

        # 如果关键词列表中包含目标关键词
        if target_keyword.lower() in [k.lower() for k in keywords]:
            # 更新共现计数
            for i, kw1 in enumerate(keywords):
                for j, kw2 in enumerate(keywords):
                    if i != j:  # 避免自身与自身的关系
                        keyword_relations[kw1.lower()][kw2.lower()] += 1

    # 检查目标关键词是否被找到
    if target_keyword.lower() not in keyword_relations:
        print(f"在数据集中未找到关键词 '{target_keyword}'")
        return None

    # 获取与目标关键词共现最多的关键词
    related_keywords = keyword_relations[target_keyword.lower()]
    print(f"找到与 '{target_keyword}' 共现的关键词数量: {len(related_keywords)}")

    # 过滤掉共现次数低于阈值的关键词
    filtered_related = {k: v for k, v in related_keywords.items() if v >= min_co_occurrence}
    print(f"共现次数 >= {min_co_occurrence} 的关键词数量: {len(filtered_related)}")

    # 获取前N个最相关的关键词
    top_keywords = [k for k, _ in sorted(filtered_related.items(), key=lambda x: x[1], reverse=True)[:top_related]]

    # 添加目标关键词
    top_keywords = [target_keyword.lower()] + [k for k in top_keywords if k != target_keyword.lower()]

    # 构建节点和边
    nodes = []
    links = []

    # 添加目标关键词作为中心节点
    nodes.append({
        "id": target_keyword.lower(),
        "name": target_keyword,
        "symbolSize": 50,  # 较大的节点大小
        "value": sum(related_keywords.values()),  # 值为总共现次数
        "category": 0,  # 中心节点类别
        "itemStyle": {"color": "#FF0000"}  # 红色
    })

    # 添加相关关键词节点
    for i, kw in enumerate(top_keywords[1:], 1):  # 跳过中心节点
        # 计算节点大小（基于共现次数）
        size = max(20, min(40, 15 + filtered_related[kw] * 2))

        nodes.append({
            "id": kw,
            "name": kw,
            "symbolSize": size,
            "value": filtered_related[kw],
            "category": 1,  # 关联节点类别
        })

        # 添加与中心节点的连接
        links.append({
            "source": target_keyword.lower(),
            "target": kw,
            "value": filtered_related[kw]
        })

    # 添加关键词之间的连接
    for i, kw1 in enumerate(top_keywords[1:], 1):
        for j, kw2 in enumerate(top_keywords[1:], 1):
            if i < j:  # 避免重复连接
                co_occurrence = keyword_relations[kw1][kw2]
                if co_occurrence >= min_co_occurrence:
                    links.append({
                        "source": kw1,
                        "target": kw2,
                        "value": co_occurrence
                    })

    # 创建类别
    categories = [
        {"name": "中心关键词"},
        {"name": "相关关键词"}
    ]

    # 创建ECharts图表
    c = (
        Graph(init_opts=opts.InitOpts(width="900px", height="700px", theme=ThemeType.LIGHT))
        .add(
            "",
            nodes=nodes,
            links=links,
            categories=categories,
            layout="force",
            is_roam=True,
            is_focusnode=True,
            is_rotate_label=True,
            is_draggable=True,
            linestyle_opts=opts.LineStyleOpts(width=1.5, curve=0.3, opacity=0.8),
            label_opts=opts.LabelOpts(is_show=True, position="right", font_size=12),
            repulsion=800,
            edge_symbol=["none", "arrow"],
            gravity=0.2,
            edge_length=120,
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=f"关键词 '{target_keyword}' 关系图"),
            tooltip_opts=opts.TooltipOpts(
                trigger="item",
                formatter="{a} <br/>{b} : {c}"
            ),
            legend_opts=opts.LegendOpts(
                orient="vertical",
                pos_left="2%",
                pos_top="20%",
                is_show=True
            )
        )
    )

    # 保存为HTML文件
    output_file = os.path.join(output_dir, f"{target_keyword.replace(' ', '_')}_relation_graph.html")
    c.render(output_file)
    print(f"关系图已保存至: {output_file}")

    # 在Jupyter中显示
    IFrame(output_file, width=1000, height=800)

    return output_file

In [17]:
# 分析单个关键词的函数
def analyze_keyword(df, keyword, output_dir):
    """分析单个关键词并生成关系图"""
    print(f"\n{'='*50}\n开始分析关键词: {keyword}\n{'='*50}")

    # 创建关键词关系图
    graph_file = create_keyword_graph(df, keyword, output_dir)

    if graph_file:
        return IFrame(graph_file, width=1000, height=800)
    else:
        return None

In [18]:
# 分析关键词列表的函数
def analyze_keywords(df, keywords_list, output_dir):
    """依次分析多个关键词并生成关系图"""
    results = {}

    for keyword in keywords_list:
        result = analyze_keyword(df, keyword, output_dir)
        results[keyword] = result

    return results

In [19]:
# 对单个关键词进行分析的函数
def run_analysis_for_keyword():
    """交互式输入一个关键词并进行分析"""
    keyword = input("请输入要分析的关键词: ")
    if keyword.strip():
        return analyze_keyword(df, keyword.strip(), output_dir)
    else:
        print("未输入关键词")
        return None

In [20]:
# 运行单个关键词分析
result = run_analysis_for_keyword()

KeyboardInterrupt: Interrupted by user

In [21]:
for keyword in insight_keywords:
    print(f"\n正在分析关键词: {keyword}")
    result = analyze_keyword(df, keyword, output_dir)
    display(result)  # 在Jupyter中显示结果


正在分析关键词: continuous fibers

开始分析关键词: continuous fibers

开始为关键词 'continuous fibers' 创建关系图...
找到与 'continuous fibers' 共现的关键词数量: 122
共现次数 >= 2 的关键词数量: 20
关系图已保存至: ./results/CFpathPlanning101_replaced_synonyms_20250512_11\continuous_fibers_relation_graph.html



正在分析关键词: path planning method

开始分析关键词: path planning method

开始为关键词 'path planning method' 创建关系图...
找到与 'path planning method' 共现的关键词数量: 185
共现次数 >= 2 的关键词数量: 27
关系图已保存至: ./results/CFpathPlanning101_replaced_synonyms_20250512_11\path_planning_method_relation_graph.html



正在分析关键词: robot programming

开始分析关键词: robot programming

开始为关键词 'robot programming' 创建关系图...
找到与 'robot programming' 共现的关键词数量: 235
共现次数 >= 2 的关键词数量: 27
关系图已保存至: ./results/CFpathPlanning101_replaced_synonyms_20250512_11\robot_programming_relation_graph.html



正在分析关键词: optimization

开始分析关键词: optimization

开始为关键词 'optimization' 创建关系图...
找到与 'optimization' 共现的关键词数量: 73
共现次数 >= 2 的关键词数量: 9
关系图已保存至: ./results/CFpathPlanning101_replaced_synonyms_20250512_11\optimization_relation_graph.html



正在分析关键词: topology optimisation

开始分析关键词: topology optimisation

开始为关键词 'topology optimisation' 创建关系图...
找到与 'topology optimisation' 共现的关键词数量: 41
共现次数 >= 2 的关键词数量: 7
关系图已保存至: ./results/CFpathPlanning101_replaced_synonyms_20250512_11\topology_optimisation_relation_graph.html
